In [ ]:
#Following command is to unzip the uploaded zip files 
#!unzip "/content/speech-emotion-recognition-ravdess-data.zip" -d "/content/data/"



In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (202 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to 

In [ ]:
pip install pyaudio

  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp37-cp37m-linux_x86_64.whl size=52614 sha256=579ef43308ff9c325e6c1d3d8a280d3e3b642e9f7f52e130fdfa4eeeb136977c
  Stored in directory: /root/.cache/pip/wheels/40/2e/4c/b71e7e96c861a46e6213bc6bb482b94dcf293a92c5e736c1ec
Successfully built pyaudio


In [ ]:
import librosa as li
import numpy as np
import soundfile as so
import sklearn as sk
import pyaudio as py

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy','angry']

In [ ]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
           continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    
    x_norm = (x - np.mean(x, axis = 0)) / np.std(x, axis = 0)
    return train_test_split( np.array(x_norm), y, test_size=test_size, random_state=9)

In [ ]:
# load RAVDESS dataset
X_train, X_test, y_train, y_test = load_data()

# print some details_
print(X_train)
print(y_train)

# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])

# number of features used
# this is a vector of features extracted 
print("[+] Number of features:", X_train.shape[1])

model_params = {
    'alpha': 0.05,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (400,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
model = MLPClassifier(**model_params)
model_2 = SVC(gamma='scale',max_iter=500, verbose=0,kernel='rbf')

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)
model_2.fit(X_train,y_train)

y_pred = model.predict(X_test)
y2_pred = model_2.predict(X_test)

# calculate the accuracy
accuracy_test = accuracy_score(y_true=y_test, y_pred=y_pred)
accuracy2_test = accuracy_score(y_true=y_test, y_pred=y2_pred)

print("MLP_Accuracy_test: {:.2f}%".format(accuracy_test*100))
print("SVC_Accuracy_test: {:.2f}%".format(accuracy2_test*100))

# save the model
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

[[-1.53231106  0.97649648  2.05530561 ... -0.43933922 -0.38812868
  -0.38885449]
 [-0.27464601  1.61600612  0.23269378 ... -0.38712241 -0.34444288
  -0.36122703]
 [ 0.26461732  0.1635458   0.97922874 ... -0.19683084 -0.22209977
  -0.27699305]
 ...
 [ 0.89978093 -1.04204285 -1.15820982 ... -0.34175526 -0.32875941
  -0.32007317]
 [-0.02812381 -0.81587709 -0.32133688 ... -0.3060116  -0.24523744
  -0.18378449]
 [-1.17618573  0.04399322  0.50810288 ... -0.41218631 -0.36992459
  -0.36997293]]
['calm', 'calm', 'angry', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'calm', 'angry', 'happy', 'calm', 'calm', 'happy', 'happy', 'happy', 'happy', 'calm', 'angry', 'calm', 'happy', 'happy', 'calm', 'angry', 'calm', 'happy', 'calm', 'angry', 'calm', 'happy', 'happy', 'happy', 'calm', 'calm', 'happy', 'angry', 'happy', 'happy', 'happy', 'happy', 'happy', 'angry', 'angry', 'angry', 'angry', 'angry', 'happy', 'angry', 'happy', 'happy', 'calm', 'angry', 'happy', 'angry', 'happy', 'calm', 'angry', 

In [ ]:
data = {'MLP_Accuracy_test':accuracy_test, 'SVC_Accuracy_test':accuracy2_test}
Classifiers = list(data.keys())
values = list(data.values())

fig = plt.figure(figsize=(10,5))

plt.bar(Classifiers, values, color=['green','blue'],width=0.4)

plt.xlabel("Classifiers")
plt.ylabel("Accuracy Obtained")
plt.title("Comparision of Classifiers")
plt.show()

NameError: ignored

In [ ]:
import torch
import torch.cuda as cuda
import torch.nn as nn
import os

# Set proxy in case it's not already set in our environment before loading torchvision

os.environ['https_proxy'] = 'http://192.41.170.23:3128'
os.environ['http_proxy'] = 'http://192.41.170.23:3128'

import torchvision

from torch.autograd import Variable

from torchvision import datasets
from torchvision import transforms

# The functional module contains helper functions for defining neural network layers as simple functions
import torch.nn.functional as F

In [ ]:
mean = 0.0
stddev = 1.0

# Transform to apply to input images


# Datasets
x_train = torch.tensor(X_train.max(axis=0))
x_test = torch.tensor(X_test.max(axis=0))

batch_size = 1024
X_train_loader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, shuffle=True, num_workers=1)
X_valid_loader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, shuffle=True, num_workers=1)


In [ ]:
# 1D CNN Approach 
class CNN_Model(nn.Module):
    
    def __init__(self):
        super().__init__()
               
        # NOTE: All Conv2d layers have a default padding of 0 and stride of 1,
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)      # 24 x 24 x 20  (after 1st convolution)
        self.relu1 = nn.ReLU()                            # Same as above
        
        # Convolution Layer 2
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)     # 20 x 20 x 20  (after 2nd convolution)
        #self.conv2_drop = nn.Dropout2d(p=0.5)            # Dropout is a regularization technqiue we discussed in class
        self.maxpool2 = nn.MaxPool2d(2)                   # 10 x 10 x 20  (after pooling)
        self.relu2 = nn.ReLU()                            # Same as above 
        
        # Fully connected layers
        self.fc1 = nn.Linear(2000, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        
        # Convolution Layer 1                    
        x = self.conv1(x)                        
        x = self.relu1(x)                        
        
        # Convolution Layer 2
        x = self.conv2(x)               
        #x = self.conv2_drop(x)
        x = self.maxpool2(x)
        x = self.relu2(x)
        
        # Switch from activation maps to vectors
        x = x.view(-1, 2000)
        
        # Fully connected layer 1
        x = self.fc1(x)
        x = F.relu(x)
        #x = F.dropout(x, training=True)
        
        # Fully connected layer 2
        x = self.fc2(x)
        
        return x

In [ ]:
# The model
net = CNN_Model()

device = 'cuda:2'#change the GPU if you want from 0 to 4, also substitute the device eveywhere you see cuda(),to see the process in terminal nvdia-smi
if cuda.is_available():
    net = net.cuda(device)

# Our loss function
criterion = nn.CrossEntropyLoss()

# Our optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9) 

In [ ]:
num_epochs = 20

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []

for epoch in range(num_epochs):
    
    ############################
    # Train
    ############################
    
    iter_loss = 0.0
    correct = 0
    iterations = 0
    
    net.train()                   # Put the network into training mode
    
    for i, (items, classes) in enumerate(X_train_loader):
        
        # Convert torch tensor to Variable
        items = Variable(items)
        classes = Variable(classes)
        
        # If we have GPU, shift the data to GPU
        if cuda.is_available():
            items = items.cuda(device)
            classes = classes.cuda(device)
        
        optimizer.zero_grad()     # Clear off the gradients from any past operation
        outputs = net(items)      # Do the forward pass
        loss = criterion(outputs, classes) # Calculate the loss
        iter_loss += loss.item() # Accumulate the loss
        loss.backward()           # Calculate the gradients with help of back propagation
        optimizer.step()          # Ask the optimizer to adjust the parameters based on the gradients
        
        # Record the correct predictions for training data 
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        iterations += 1
    
    # Record the training loss
    train_loss.append(iter_loss / iterations)
    # Record the training accuracy
    train_accuracy.append((100 * correct.item() / float(len(X_train_loader.dataset))))
   

    ############################
    # Validate - How did we do on the unseen dataset?
    ############################
    
    loss = 0.0
    correct = 0
    iterations = 0

    net.eval()                    # Put the network into evaluate mode
    
    for i, (items, classes) in enumerate(X_valid_loader):
        
        # Convert torch tensor to Variable
        items = Variable(items)
        classes = Variable(classes)
        
        # If we have GPU, shift the data to GPU
        if cuda.is_available():
            items = items.cuda(device)
            classes = classes.cuda(device)
        
        outputs = net(items)      # Do the forward pass
        loss += criterion(outputs, classes).item() # Calculate the loss
        
        # Record the correct predictions for training data
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations += 1

    # Record the validation loss
    valid_loss.append(loss / iterations)
    # Record the validation accuracy
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar.item() / len(X_valid_loader.dataset) * 100.0)

    print ('Epoch %d/%d, Tr Loss: %.4f, Tr Acc: %.4f, Val Loss: %.4f, Val Acc: %.4f'
           %(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1], 
             valid_loss[-1], valid_accuracy[-1]))
    

ValueError: ignored

In [ ]:
f = plt.figure(figsize=(10, 8))
plt.plot(train_loss, label='training loss')
plt.plot(valid_loss, label='validation loss')
plt.title('MNIST loss, conv-5x5x10-relu-conv-5x5x20-maxpool-2x2-relu-fc50-fc10')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

f = plt.figure(figsize=(10, 8))
plt.plot(train_accuracy, label='training accuracy')
plt.plot(valid_accuracy, label='validation accuracy')
plt.legend()
plt.show()